In [1]:
import numpy as np
import pandas as pd
import unicodedata
import re
import matplotlib as plt
import seaborn as sns
import nltk 

# Uncomment to see full cell text
pd.set_option('display.max_colwidth', 0)

raw = pd.read_excel("../data/raw/inquire-boulder-faq-text/inquire-boulder-active-faqs-2019-01-02.xls.xlsx")
faq = raw.drop(labels=('Active'), axis=1).dropna()

Topic         Drivers License and License Plates                                                                                                            
Category      Legal                                                                                                                                         
Department    City Attorney                                                                                                                                 
FAQ           To register a motor vehicle or obtain a drivers license or license plate, visit the Boulder County Motor Vehicle website or call 303-413-7710.
Updated       2013-06-21 00:00:00                                                                                                                           
Name: 92, dtype: object

In [2]:
#raw.set_index(keys=['Department', 'Category', 'Topic']).sort_index().drop(labels=('Active'), axis=1).dropna()

In [3]:
#faq.length.plot(kind='hist', bins=50)

In [2]:
def find_heading(row):
    entry = row['FAQ']
    if entry.find('?') < 0:
        return entry
    q = entry.split('?')[0].split('. ')[-1]
    heading = entry.split(q)[0]
    return heading

def strip_heading(row):
    return row['FAQ'].replace(row['heading'], '')

def find_questions(entry):
    questions = []
    qna = entry.split('?')
    for x in qna:
        if x.find('. ') < 0:
            q = x
        else:
            q = x.split('. ')[-1]
        if not q.isspace() and q:
            questions.append(q.strip())
    return questions

def find_answers(entry):
    answers = []
    a = ''
    qna = entry.split('?')
    for x in qna:
        if x.find('. ') < 0:
            continue
        else:
            a = x.replace(x.split('. ')[-1], '')
        if not a.isspace() and a:
                answers.append(a.strip())
    return answers

def preprocess(entry):
    entry = entry.replace('\n', ' ')
    entry = unicodedata.normalize("NFKD", entry)
    entry = entry.replace('?', '? ')
    re.sub(r'.(?=[A-Z])', '. ', entry)
    return entry + ' '

# Pick out questions-and-answer pairs with a regular expression
def qna_pairs(row):
    l = 0
    relist = []
    match = re.findall(r'([A-Z][^.]*\?\s)([^?]+(?:\.\s|\.\"|\.(?=[A-Z])|\)(?=I)|[a-z](?=[A-Z])))', row.FAQ)
    if(match):
        for s in match:
            relist.append([s[0], s[1]])
            l += len(s[0])
            l += len(s[1])
#             print('-----------------')
#             print(s[0])
#             print('****')
#             print(s[1])            
#         print("%d: diff length %d\t%s" % (row.name, len(row.FAQ) - l, row.Topic))
        row['qna'] = relist
    else:
        row['qna'] = [[row['Topic'], row['heading']]]
#         print("%d: NA\t\t\t%s" % (row.name, row.Topic))
        l
    return row

def process_text(df):
    df.FAQ = df.FAQ.apply(preprocess)
    df['heading'] = df.apply(find_heading, axis=1)
    df['FAQ'] = df.apply(strip_heading, axis=1)
    df = df.apply(qna_pairs, axis=1)
    return df
    
faq = process_text(faq)

In [7]:
# faq['questions'] = faq.FAQ.apply(find_questions)
# faq['answers'] = faq.FAQ.apply(find_answers)

# def count_qna(row):
#     print('%d:\t%d\t%d' % (row.name, len(row.questions), len(row.answers)))
#     return abs(len(row.questions) - len(row.answers))
# s = faq.apply(count_qna, axis=1)
# s.sum()

In [3]:
s = faq.apply(lambda x: pd.Series(x['qna']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'question'
faq = faq.drop(['FAQ', 'qna'], axis=1).join(s).reset_index(drop=True)
faq['answer'] = faq.question.apply(lambda x: x[1])
faq['question'] = faq.question.apply(lambda x: x[0])
faq

,Topic,Category,Department,Updated,heading,question,answer
0,Affordable Homeownership,Housing,Housing,2015-06-19,"The City of Boulder is committed to making housing in our community available to a variety of people, including those who could not otherwise purchase a home in Boulder's housing market. The Division of Housing offers opportunities for homeownership to those with low and moderate incomes - from down payment assistance on market rate homes to homes that must be sold at affordable prices.",How long does it take to become income-certified?,Generally up to two weeks.
1,Affordable Homeownership,Housing,Housing,2015-06-19,"The City of Boulder is committed to making housing in our community available to a variety of people, including those who could not otherwise purchase a home in Boulder's housing market. The Division of Housing offers opportunities for homeownership to those with low and moderate incomes - from down payment assistance on market rate homes to homes that must be sold at affordable prices.",Who needs to be on the application?,"Everyone who lives in the household or is part of the household, in addition to all people who will be listed on the property title. Household members should include all individuals: legal spouse, domestic partner, or common-law spouse; children(under 18 who reside with the applicant at least 50 percent of the time; and/or a significant other whom the applicant chooses to include as a member of his/her household, who will be occupying the house. The applicant's spouse must be included unless they are legally divorced or separated. Persons not counted include foster children, unborn children, and children under 18 who reside with the applicant less then 50 percent of the time."
2,Affordable Homeownership,Housing,Housing,2015-06-19,"The City of Boulder is committed to making housing in our community available to a variety of people, including those who could not otherwise purchase a home in Boulder's housing market. The Division of Housing offers opportunities for homeownership to those with low and moderate incomes - from down payment assistance on market rate homes to homes that must be sold at affordable prices.",How is annual income calculated?,"Annual income is defined as the anticipated total income for the next 12-month period received from all sources by each member (over the age of 18) of the household. It is assumed that today's circumstances will continue for the next 12 months. The applicant is required to verify this by either submitting a letter from his/her employer stating his/her annual wage or by completing an employer verification form. To obtain this form, call the Housing Division at 303-441-3157 ext. 2."
3,Affordable Homeownership,Housing,Housing,2015-06-19,"The City of Boulder is committed to making housing in our community available to a variety of people, including those who could not otherwise purchase a home in Boulder's housing market. The Division of Housing offers opportunities for homeownership to those with low and moderate incomes - from down payment assistance on market rate homes to homes that must be sold at affordable prices.",How is income verified for self-employed people?,Self-employed individuals should submit copies of their last three years of federal income-tax forms and must also submit an up-to-date profit/loss statement.
4,Affordable Homeownership,Housing,Housing,2015-06-19,"The City of Boulder is committed to making housing in our community available to a variety of people, including those who could not otherwise purchase a home in Boulder's housing market. The Division of Housing offers opportunities for homeownership to those with low and moderate incomes - from down payment assistance on market rate homes to homes that must be sold at affordable prices.",How are assets calculated?,"Assets are calculated based on annual income earned (interest), not the value of the asset itself."
5,Affordable Homeownership,Housing,Housing,2015-06-19,"

# NLP

In [14]:
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
#nltk.download('stopwords')

In [15]:
def text_process(text):
    # Remove punctuation
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)
    
    # Remove stopwords
    return [word.lower() for word in nopunc.split() if word.lower() not in stopwords.words('english')]

In [39]:
faq['FAQ'].head().apply(text_process)

0    [city, boulder, committed, making, housing, community, available, variety, people, including, could, otherwise, purchase, home, boulders, housing, market, division, housing, offers, opportunities, homeownership, low, moderate, incomes, payment, assistance, market, rate, homes, homes, must, sold, affordable, prices, long, take, become, incomecertifiedgenerally, two, weeks, needs, applicationeveryone, lives, household, part, household, addition, people, listed, property, title, household, members, include, individuals, legal, spouse, domestic, partner, commonlaw, spouse, childrenunder, 18, reside, applicant, least, 50, percent, time, andor, significant, applicant, chooses, include, member, hisher, household, occupying, house, applicants, spouse, must, included, unless, legally, divorced, separated, persons, counted, include, foster, children, unborn, children, children, 18, reside, applicant, less, ...]                                                                             
1  

In [40]:
# Might take a while...
bow_transformer = CountVectorizer(analyzer=text_process).fit(faq.FAQ)

# Print total number of vocab words
print(len(bow_transformer.vocabulary_))

7054


In [41]:
# Let's take one text message and get its bag-of-words counts as a vector, putting to use our new `bow_transformer`:
entry = faq['FAQ'][0]
print(entry)

The City of Boulder is committed to making housing in our community available to a variety of people, including those who could not otherwise purchase a home in Boulder's housing market. The Division of Housing offers opportunities for homeownership to those with low and moderate incomes - from down payment assistance on market rate homes to homes that must be sold at affordable prices. How long does it take to become income-certified?Generally up to two weeks. Who needs to be on the application?Everyone who lives in the household or is part of the household, in addition to all people who will be listed on the property title. Household members should include all individuals: legal spouse, domestic partner, or common-law spouse; children(under 18 who reside with the applicant at least 50 percent of the time; and/or a significant other whom the applicant chooses to include as a member of his/her household, who will be occupying the house. The applicant's spouse must be included unless th

In [42]:
# Now let's see its vector representation:
bow = bow_transformer.transform([entry])
print(bow)
print(bow.shape)

  (0, 26)	1
  (0, 37)	1
  (0, 47)	1
  (0, 61)	3
  (0, 86)	1
  (0, 87)	1
  (0, 147)	1
  (0, 165)	1
  (0, 218)	1
  (0, 319)	2
  (0, 538)	1
  (0, 542)	1
  (0, 608)	5
  (0, 614)	1
  (0, 665)	1
  (0, 672)	1
  (0, 673)	1
  (0, 711)	1
  (0, 728)	3
  (0, 737)	1
  (0, 754)	1
  (0, 757)	4
  (0, 758)	1
  (0, 760)	1
  (0, 762)	1
  :	:
  (0, 6098)	1
  (0, 6185)	2
  (0, 6189)	1
  (0, 6198)	1
  (0, 6277)	1
  (0, 6393)	3
  (0, 6417)	2
  (0, 6428)	2
  (0, 6432)	1
  (0, 6451)	1
  (0, 6567)	1
  (0, 6583)	1
  (0, 6624)	2
  (0, 6656)	1
  (0, 6666)	1
  (0, 6703)	1
  (0, 6716)	1
  (0, 6738)	1
  (0, 6739)	1
  (0, 6740)	1
  (0, 6784)	1
  (0, 6809)	1
  (0, 6865)	1
  (0, 6877)	1
  (0, 7005)	3
(1, 7054)


This means that there are 8 unique words in message number 0 (after removing common stop words). One of them appears twice, the rest only once. Let's go ahead and check and confirm which one appears twice:

In [44]:
print(bow_transformer.get_feature_names()[7005])

years


Now we can use .transform on our Bag-of-Words (bow) transformed object and transform the entire DataFrame. Let's go ahead and check out how the bag-of-words counts for the entire FAQ corpus is a large, sparse matrix:

In [45]:
faq_bow = bow_transformer.transform(faq['FAQ'])
print('Shape of Sparse Matrix: ', faq_bow.shape)
print('Amount of Non-Zero occurences: ', faq_bow.nnz)

Shape of Sparse Matrix:  (250, 7054)
Amount of Non-Zero occurences:  24520


In [46]:
sparsity = (100.0 * faq_bow.nnz / (faq_bow.shape[0] * faq_bow.shape[1]))
print('sparsity: {}% Non-zero values'.format(round(sparsity)))

sparsity: 1% Non-zero values


In [47]:
tfidf_transformer = TfidfTransformer().fit(faq_bow)
tfidf = tfidf_transformer.transform(bow)
print(tfidf)

  (0, 7005)	0.09582936818073878
  (0, 6877)	0.03681836557246331
  (0, 6865)	0.024657504319853925
  (0, 6809)	0.05202494620510897
  (0, 6784)	0.020971819727887548
  (0, 6740)	0.04584198079252669
  (0, 6739)	0.04584198079252669
  (0, 6738)	0.05202494620510897
  (0, 6716)	0.03405174219430081
  (0, 6703)	0.04222517788291014
  (0, 6666)	0.029145255416496887
  (0, 6656)	0.043851510516499624
  (0, 6624)	0.06695209993472429
  (0, 6583)	0.05202494620510897
  (0, 6567)	0.029145255416496887
  (0, 6451)	0.04222517788291014
  (0, 6432)	0.04840814329549242
  (0, 6428)	0.09168396158505338
  (0, 6417)	0.04525128247702502
  (0, 6393)	0.09582936818073878
  (0, 6277)	0.029145255416496887
  (0, 6198)	0.04840814329549242
  (0, 6189)	0.039659015379944415
  (0, 6185)	0.05515257436685294
  (0, 6098)	0.04840814329549242
  :	:
  (0, 762)	0.05202494620510897
  (0, 760)	0.026752304832442845
  (0, 758)	0.03532822082907916
  (0, 757)	0.1506741804156694
  (0, 754)	0.043851510516499624
  (0, 737)	0.04840814329549242


In [48]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['boulder']])
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['county']])

1.2221480310727082
2.861891293002137


In [49]:
faq_tfidf = tfidf_transformer.transform(faq_bow)
print(faq_tfidf.shape)

(250, 7054)


### Train the model

In [50]:
from sklearn.naive_bayes import MultinomialNB
faq_match_model = MultinomialNB().fit(faq_tfidf, faq['Topic'])

In [51]:
print('predicted:', faq_match_model.predict(tfidf)[0])
print('expected:', faq.Topic[0])

predicted: Affordable Homeownership
expected: Affordable Homeownership


### Evaluate the model

In [58]:
pred = faq_match_model.predict(faq_tfidf)
print(pred)

['Affordable Homeownership' 'Affordable Rental Opportunities'
 'Aircraft Noise' 'Airport' 'Alley Maintenance'
 'Americans for the Arts Convention Scholarships'
 'Americans with Disabilities Act (ADA)'
 'Americans with Disabilities Act (ADA) Trails'
 'Animal Noise (Barking Dogs)' 'Animal Protection, Rules and Regulations'
 'Animal Shelter' 'Ask-a-Librarian' 'Auction Licensing Instructions'
 'Backflow Prevention - Drinking Water'
 'Bear Resistant Container Required' 'Bears' 'Bicycling'
 'Birth and Death Certificate Request' 'Block Party'
 'Boulder Arts Commission Grants' 'Boulder City Limits'
 'Boulder Civic Area' 'Boulder Convention and Visitors Bureau'
 'Boulder Measures' 'Boulder Office of Emergency Management'
 'Boulder Planning' 'Boulder Reservoir' 'Boulder Revised Code'
 'BoulderReads!' 'Broken Parking Meter or Pay Station' 'Building Permits'
 'Building Safety Code Violations' 'Bulk Water Sales'
 'Business Licensing Questions' 'Business Services'
 'Canyon Gallery and Art Exhibit Sp

In [59]:
from sklearn.metrics import classification_report
print (classification_report(faq['Topic'], pred))

                                                                      precision    recall  f1-score   support

                                                  Accessible Parking       1.00      1.00      1.00         1
                                            Affordable Homeownership       1.00      1.00      1.00         1
                                     Affordable Rental Opportunities       1.00      1.00      1.00         1
                                                      Aircraft Noise       1.00      1.00      1.00         1
                                                             Airport       1.00      1.00      1.00         1
                                                   Alley Maintenance       1.00      1.00      1.00         1
                      Americans for the Arts Convention Scholarships       1.00      1.00      1.00         1
                               Americans with Disabilities Act (ADA)       1.00      1.00      1.00         1
         

In [60]:
from sklearn.model_selection import train_test_split

faq_train, faq_test, topic_train, topic_test = \
train_test_split(faq['FAQ'], faq['Topic'], test_size=0.2)

print(len(faq_train), len(faq_test), len(faq_train) + len(faq_test))

200 50 250


In [86]:
from sklearn.pipeline import Pipeline

tfidf_transformer = TfidfTransformer().fit(faq_bow)

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    ('tfidf', tfidf_transformer),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])

In [87]:
pipeline.fit(faq_train,topic_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function text_process at 0x1166e5ea0>, binary=False,
        decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocessor=None,...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [88]:
pred = pipeline.predict(faq_test)

In [113]:
#print(classification_report(pred, faq_test))